In [1]:
!pip install pyspark pandas nltk kagglehub pyngrok kaggle synapseml==0.11.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 550.0/550.0 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 9.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747859 sha256=ec7f21a50f20c22ce5161e4fce642e9c31d1855a9826785440dd415b3a8f78ab
  Stored in directory: /root/.cache/pip/wheels/0c/7f/b4/0e68c6d8d89d2e582e5498ad88616c16d7c19028680e9d3840
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyngrok import ngrok
import os
from pyspark.sql.functions import col, lower, regexp_replace, udf
from pyspark.sql.types import ArrayType, StringType, IntegerType
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pyspark.sql.functions import col, isnan, when
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.linalg import DenseVector
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


nltk.download("stopwords")
nltk.download("punkt")
nltk.download('punkt_tab')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("SparkBenchmark").getOrCreate()

print(spark)


In [ ]:
#!ngrok authtoken xxxxxxx
#configuration = (SparkSession.builder
#         .appName("NYC Taxi Price Prediction")
#         .config("spark.driver.memory", "100g")
#         .config("spark.executor.memory", "50g")
#         .config("spark.executor.cores", "4")
#         .config("spark.dynamicAllocation.enabled", "true")
#         .config("spark.dynamicAllocation.minExecutors", "2")
#         .config("spark.dynamicAllocation.maxExecutors", "10")
#         .config("spark.sql.shuffle.partitions", "20")
#      )

#spark = configuration.getOrCreate()
#print("Spark initialized!")

In [4]:
def download_and_extract(dataset, file_name):
    download_path = "./downloads"
    zip_file_path = file_name + ".zip"
    command = f"kaggle datasets download {dataset} --file {file_name}"
    os.system(command)

    os.system(f"unzip {zip_file_path} -d {download_path}")
    extracted_file_path = os.path.join(download_path, file_name)

    return os.path.join(download_path, file_name)

def preprocess_text(text):
    # Tokenize
    tokens = word_tokenize(text.lower() if text else "")
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words and len(word) > 2]
    return tokens

In [9]:
import os
dataset = "cynthiarempel/amazon-us-customer-reviews-dataset"
file_name = "amazon_reviews_us_Wireless_v1_00.tsv"

file_path = download_and_extract(dataset, file_name)
spark_df = spark.read.csv(
    file_path,
    sep="\t",
    header=True,
    inferSchema=True,
    multiLine=True
).sample(fraction=0.1, seed=42)

print(f"Dataset loaded successfully with {spark_df.count()} rows.")
spark_df.show(5)


Dataset loaded successfully with 900773 rows.
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48086021|R3MWLXLNO21PDQ|B00IP1MQNK|     488006702|Lumsing 10400mah ...|        Wireless|          5|            0|          0|   N|                Y|          Five Stars|         Works great| 2015-08-31|
|         US|   12218556|R26VY1L1FD3LPU|B00BJN45GM|      47788188|OtterBox

In [13]:
preprocess_udf = udf(preprocess_text, ArrayType(StringType()))

preprocessed_df = spark_df \
    .withColumn("review_body", lower(col("review_body"))) \
    .withColumn("review_body", regexp_replace(col("review_body"), r"http\S+|www\S+|https\S+", "")) \
    .withColumn("review_body", regexp_replace(col("review_body"), r"[^a-zA-Z\s]", "")) \
    .withColumn("review_body", regexp_replace(col("review_body"), r"\s+", " ")) \
    .withColumn("tokens", preprocess_udf(col("review_body"))) \
    .withColumn("sentiment", (col("star_rating") > 3).cast(IntegerType()))

preprocessed_df = preprocessed_df.filter(col("star_rating").isNotNull())
preprocessed_df = preprocessed_df.withColumn("sentiment", (col("star_rating") > 3).cast(IntegerType()))
preprocessed_df = preprocessed_df.filter(col("sentiment").isNotNull() & ~isnan(col("sentiment")))


preprocessed_df.show(5)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-----------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------------------------------------------------------------------------------------------------------------------------+---------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                                                 |product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline                                      |review_body                                                    

In [16]:
hashing_tf = HashingTF(inputCol="tokens", outputCol="raw_features", numFeatures=300)
tf_df = hashing_tf.transform(preprocessed_df)

idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(tf_df)
final_df = idf_model.transform(tf_df)

final_df = final_df.select("features", "sentiment")
final_df.show(5)


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|features                                                                                                                                                                                                                                                                                                                                            |sentiment|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#from pyspark.ml.feature import CountVectorizer, IDF

#cv = CountVectorizer(inputCol="tokens", outputCol="rawFeatures", vocabSize=1000, minDF=5)

# Fit the CountVectorizer model and transform the data:
#cv_model = cv.fit(preprocessed_df)
#tf_df = cv_model.transform(preprocessed_df)

In [17]:
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)

In [19]:
rf = RandomForestClassifier(labelCol="sentiment", featuresCol="features", numTrees=10, maxDepth=6,
                           featureSubsetStrategy="auto", subsamplingRate=0.8)

train_df.cache()

rf_model = rf.fit(train_df)

predictions = rf_model.transform(test_df)
evaluator = BinaryClassificationEvaluator(labelCol="sentiment", metricName="areaUnderROC")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.7391002704451265
